# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

## Important Note:

- import hvplot.pandas - I tried desperately to get this module to work but couldn't.
- I reached out to AskBCS and TAs in office hours, TA replicated the issue and suggested using alternate module.
- There appears to be module conflicts within hvplot latest version.
- I solved the challenge using plotly.express

In [81]:
# Dependencies and Setup
import pandas as pd
## import hvplot.pandas - I tried desperately to get this module to work but couldn't.
## I reached out to AskBCS and TAs in office hours, TA replicated the issue and suggested using alternate module.
## There appears to be module conflicts within hvplot latest version.
import plotly.express as px
import requests
import pandas as pd
import random

# Import API key
from api_keys import geoapify_key

# Define your preferred weather conditions
ideal_conditions = {
    "temperature": 25, # Celsius
    "humidity": 50, # Percentage
    "wind_speed": 10 # Meters per second
}



In [44]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-87.889731,-76.086121,-41.20,91,15,6.81,ar,1680639487
1,1,fort nelson,63.180001,-120.341920,-7.17,81,7,4.32,ca,1680639488
2,2,albany,-75.822862,104.119801,-29.61,99,100,3.88,au,1680639488
3,3,lashio,22.103771,97.722752,15.85,39,10,0.95,mm,1680639489
4,4,te anau,-43.080785,166.643941,16.34,87,97,8.74,nz,1680639489


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [128]:
# Use plotly.express to create a scatter mapbox of all locations on the city_data_df where marker & color signifies humidity
fig = px.scatter_mapbox(
                        city_data_df,
                        lat="Lat", lon="Lng",
                        hover_name="City",
                        hover_data=["Country", "Max Temp"],
                        size="Humidity",
                        zoom=1,
                        height=600,
                        color="Humidity",
                        color_continuous_scale=px.colors.sequential.haline
)
fig.update_layout(mapbox_style="open-street-map")
fig.show()

In [37]:
# Check the geoapify_api_url for accuracy
geoapify_api_url

'https://api.geoapify.com/v2/places?lat=-33.865143&lon=151.2099&radius=10000&type=hotel&limit=1&apiKey=e75b897acb334e9c88df1ce40734b3c4'

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [99]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df["Max Temp"] > 20) & (city_data_df["Max Temp"] < 30) & (city_data_df["Humidity"] < 50)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
57,57,wagar,22.213284,43.140888,22.98,21,99,6.51,sa,1680639506
70,70,taolanaro,16.622579,37.000776,26.08,40,100,2.64,sd,1680639510
82,82,teguldet,-37.043211,-70.740765,24.21,10,0,2.15,cl,1680639514
123,123,coquimbo,23.620755,80.426378,24.23,37,41,0.51,in,1680639527
174,174,esperance,-29.886952,136.310476,22.11,24,100,3.99,au,1680639543
317,317,ratnagiri,38.186035,-80.597304,23.81,44,10,3.30,us,1680639589
325,325,yaksatovo,26.063724,-103.019057,29.60,11,100,6.32,mx,1680639591
340,340,zhigansk,26.362442,71.336087,25.98,24,1,4.50,in,1680639596
341,341,satitoa,-31.926105,18.549387,21.42,40,0,3.77,za,1680639596
349,349,lucapa,20.670753,24.458928,25.25,9,100,4.50,td,1680639599


### Step 3: Create a new DataFrame called `hotel_df`.

In [100]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Create a DataFrame to hold the hotel data
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
57,wagar,sa,22.213284,43.140888,21,
70,taolanaro,sd,16.622579,37.000776,40,
82,teguldet,cl,-37.043211,-70.740765,10,
123,coquimbo,in,23.620755,80.426378,37,
174,esperance,au,-29.886952,136.310476,24,
317,ratnagiri,us,38.186035,-80.597304,44,
325,yaksatovo,mx,26.063724,-103.019057,11,
340,zhigansk,in,26.362442,71.336087,24,
341,satitoa,za,-31.926105,18.549387,40,
349,lucapa,td,20.670753,24.458928,9,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [102]:
# Set parameters to search for a hotel
radius = 10000 # meters
params = {
    "filter": "",
    "bias": "",
}



# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # Define the base url for the Geoapify API Hotel Search
    geoapify_api_url = f"https://api.geoapify.com/v2/places?categories=accommodation.hotel&lat={lat}&lon={lng}&radius={radius}&limit=1&apiKey={geoapify_key}"
    
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"distance.lt:{radius}"
    params["bias"] = f"circle:{radius}@{lat},{lng}"
    
    # Make API request using the f-string formatted URL and params dictionary
    hotel_data = requests.get(geoapify_api_url).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

# Check the most recent Geoapify URL for accuracy
geoapify_api_url

Starting hotel search
wagar - nearest hotel: No hotel found
taolanaro - nearest hotel: No hotel found
teguldet - nearest hotel: No hotel found
coquimbo - nearest hotel: Hostería Las Ovejas
esperance - nearest hotel: No hotel found
ratnagiri - nearest hotel: No hotel found
yaksatovo - nearest hotel: Sleep Inn Summersville
zhigansk - nearest hotel: Hotel 10
satitoa - nearest hotel: No hotel found
lucapa - nearest hotel: Klawer Hotel
longyearbyen - nearest hotel: No hotel found
kysyl-syr - nearest hotel: Hotel Tejas
broken hill - nearest hotel: No hotel found
san luis - nearest hotel: No hotel found
tilichiki - nearest hotel: No hotel found
ust-nera - nearest hotel: Hotel Park Inn


'https://api.geoapify.com/v2/places?categories=accommodation.hotel&lat=23.20106891245777&lon=84.90993968279122&radius=10000&limit=1&apiKey=e75b897acb334e9c88df1ce40734b3c4'

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [125]:
# Drop rows with no hotel
hotel_df = hotel_df[hotel_df["Hotel Name"] != "No hotel found"]

# Use plotly.express to create a scatter mapbox of all locations on the city_data_df where marker & color signifies humidity
fig = px.scatter_mapbox(
                        hotel_df,
                        lat="Lat", lon="Lng",
                        hover_name="City",
                        hover_data=["Hotel Name", "Country"],
                        size="Humidity",
                        zoom=0,
                        height=600,
                        color="Humidity",
                        color_continuous_scale=px.colors.sequential.Agsunset,
                        opacity=1
)
fig.update_layout(mapbox_style="open-street-map")
fig.show()